In [1]:
import pandas as pd
import os
import csv
from datetime import datetime
import numpy as np

# Infection and Mobility Data Validity Checks

We check whether there are negative or null values in the infection dataset us-counties-4-26.csv
and the mobility dataset DL-us-mobility-daterow-4-23.csv. The code below prints out rows with
negative or null values for the cases, deaths, mobility, and mobility index values of the two
files.

NYTimes Data

In [2]:
# Check whether cases and deaths in NYTimes data have negative values

f = open(r"us-counties-4-26.csv")
csv_f = csv.reader(f)
count = 0

for row in csv_f:
    if count == 0:
        count+=1
        continue
    if int(row[4]) < 0 or int(row[5]) < 0 or row[4] == "" or row[5] == "":
        print(row)
        
f.close()

Descartes Lab Mobility Data

In [3]:
# Check whether m50 and m50_index data have negative or null values 
f = open(r"DL-us-mobility-daterow-4-23.csv")
csv_f = csv.reader(f)
count = 0

for row in csv_f:
    if count == 0:
        count += 1
        continue
    if int(row[-1]) < 0 or float(row[-2]) < 0.0 or row[-1] == "" or row[-2] == "":
        print(row)
f.close()

# Time Series data wrangling and cleaning



NYTimes case and death records start on different dates for different counties.
Thus, must register case and death data in dataframes where columns are dates.
This is important for the time-series cross correlation analysis as scipy.correlate
will not check that the value corresponds to the correct date.
If there are missing values in the time-series data, when using scipy.correlate,
the sequence will just be truncated and shifted, producing a different waveform

The NYTimes infection and death dataset file is us-counties-4-26.csv, updated 4/26.

The Descartes mobility index data DL-us-m50_index-4-23.csv, updated 4/23,  is already 
in the desired structure and data for all counties start and end on the same dates.
However, it is actually missing data on 2020-04-20. A column of zeros is added for 2020-04-20
instead of interpolating as there there will be smoothing prior to calculting the 
cross-correlation.

DL-us-m50_index-4-23.csv provides the same data as DL-us-mobility-daterow-4-23.csv
except the data has already been transposed

Null values are replaced with zeros.

In [ ]:
# Time Series: Descartes Lab Mobility Index Data

In [4]:
#Descartes mobility index data is available between 2020-03-01 and 2020-04-23
dl = pd.read_csv('DL-us-m50_index-4-23.csv')

print('Number of rows with NULL fips: ' + str(dl.loc[dl['fips'].isnull()].shape[0]))

#NaN fips numbers converted to 0, otherwise converted to int
dl['fips'] = dl['fips'].apply(lambda x: 0 if np.isnan(x) else int(x))

#were no fips with zero values previously 
print('Number of rows with 0 fips after transformation: ' + str(dl.loc[dl['fips']==0].shape[0]))



Number of rows with NULL fips: 1
Number of rows with 0 fips after transformation: 1


In [5]:
dl.head()

,country_code,admin_level,admin1,admin2,fips,2020-03-01,2020-03-02,2020-03-03,2020-03-04,2020-03-05,...,2020-04-13,2020-04-14,2020-04-15,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-21,2020-04-22,2020-04-23
0,US,1,Alabama,NaN,1,79.0,98.0,100.0,96.0,104.0,...,50.0,51.0,56.0,59.0,68.0,49.0,4.0,62.0,62.0,47.0
1,US,2,Alabama,Autauga County,1001,49.0,100.0,95.0,95.0,100.0,...,43.0,36.0,42.0,45.0,56.0,34.0,0.0,48.0,46.0,35.0
2,US,2,Alabama,Baldwin County,1003,81.0,100.0,95.0,90.0,102.0,...,48.0,49.0,52.0,53.0,57.0,43.0,16.0,58.0,59.0,36.0
3,US,2,Alabama,Barbour County,1005,90.0,107.0,100.0,70.0,88.0,...,50.0,51.0,54.0,62.0,67.0,58.0,7.0,71.0,78.0,50.0
4,US,2,Alabama,Bibb County,1007,53.0,95.0,100.0,94.0,111.0,...,43.0,51.0,50.0,55.0,55.0,45.0,4.0,59.0,53.0,35.0


In [6]:
dl['fips'].describe()

count     2722.00000
mean     29586.14144
std      15682.17198
min          0.00000
25%      18031.50000
50%      28154.00000
75%      45016.50000
max      56041.00000
Name: fips, dtype: float64

No negative fips values, drop zero-valued rows. Also drop non-date columns, not including fips

In [7]:
dl = dl.loc[dl['fips'] != 0]
dl.drop(['country_code', 'admin_level', 'admin1', 'admin2'], axis = 1, inplace = True)
dl.head()

,fips,2020-03-01,2020-03-02,2020-03-03,2020-03-04,2020-03-05,2020-03-06,2020-03-07,2020-03-08,2020-03-09,...,2020-04-13,2020-04-14,2020-04-15,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-21,2020-04-22,2020-04-23
0,1,79.0,98.0,100.0,96.0,104.0,123.0,107.0,80.0,103.0,...,50.0,51.0,56.0,59.0,68.0,49.0,4.0,62.0,62.0,47.0
1,1001,49.0,100.0,95.0,95.0,100.0,116.0,79.0,58.0,108.0,...,43.0,36.0,42.0,45.0,56.0,34.0,0.0,48.0,46.0,35.0
2,1003,81.0,100.0,95.0,90.0,102.0,114.0,95.0,82.0,96.0,...,48.0,49.0,52.0,53.0,57.0,43.0,16.0,58.0,59.0,36.0
3,1005,90.0,107.0,100.0,70.0,88.0,118.0,141.0,79.0,98.0,...,50.0,51.0,54.0,62.0,67.0,58.0,7.0,71.0,78.0,50.0
4,1007,53.0,95.0,100.0,94.0,111.0,133.0,112.0,51.0,124.0,...,43.0,51.0,50.0,55.0,55.0,45.0,4.0,59.0,53.0,35.0


A column for 2020-04-20 is missing. Add a column of zeros. Also replace NaN's with 0

In [8]:
dl['2020-04-20'] = 0.0
dl.fillna(0, inplace = True)
dl.head()

,fips,2020-03-01,2020-03-02,2020-03-03,2020-03-04,2020-03-05,2020-03-06,2020-03-07,2020-03-08,2020-03-09,...,2020-04-14,2020-04-15,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-21,2020-04-22,2020-04-23,2020-04-20
0,1,79.0,98.0,100.0,96.0,104.0,123.0,107.0,80.0,103.0,...,51.0,56.0,59.0,68.0,49.0,4.0,62.0,62.0,47.0,0.0
1,1001,49.0,100.0,95.0,95.0,100.0,116.0,79.0,58.0,108.0,...,36.0,42.0,45.0,56.0,34.0,0.0,48.0,46.0,35.0,0.0
2,1003,81.0,100.0,95.0,90.0,102.0,114.0,95.0,82.0,96.0,...,49.0,52.0,53.0,57.0,43.0,16.0,58.0,59.0,36.0,0.0
3,1005,90.0,107.0,100.0,70.0,88.0,118.0,141.0,79.0,98.0,...,51.0,54.0,62.0,67.0,58.0,7.0,71.0,78.0,50.0,0.0
4,1007,53.0,95.0,100.0,94.0,111.0,133.0,112.0,51.0,124.0,...,51.0,50.0,55.0,55.0,45.0,4.0,59.0,53.0,35.0,0.0


# Time Series: NYTimes data 

In [9]:
nyt = pd.read_csv('us-counties-4-26.csv')

print('Number of rows with NULL fips: ' + str(nyt.loc[nyt['fips'].isnull()].shape[0]))

#NaN fips numbers converted to 0, otherwise converted to int
nyt['fips'] = nyt['fips'].apply(lambda x: 0 if np.isnan(x) else int(x))

#were no fips with zero values previously 
print('Number of rows with 0 fips after transformation: ' + str(nyt.loc[nyt['fips']==0].shape[0]))

Number of rows with NULL fips: 1097
Number of rows with 0 fips after transformation: 1097


In [10]:
#show transformed dataframe
nyt.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061,1,0
1,2020-01-22,Snohomish,Washington,53061,1,0
2,2020-01-23,Snohomish,Washington,53061,1,0
3,2020-01-24,Cook,Illinois,17031,1,0
4,2020-01-24,Snohomish,Washington,53061,1,0


In [11]:
nyt['fips'].describe()

count    89772.000000
mean     29409.773894
std      15716.079457
min          0.000000
25%      17161.000000
50%      28131.000000
75%      44001.000000
max      56043.000000
Name: fips, dtype: float64

In [12]:
#no negative fips numbers
#drop rows with fips == 0
nyt = nyt.loc[nyt['fips'] != 0]

In [13]:
temp = nyt['date'].unique().tolist()
nytdates = [datetime.strptime(x, '%Y-%m-%d') for x in temp]

firstdate = datetime.strftime(min(nytdates), '%Y-%m-%d')
lastdate = datetime.strftime(max(nytdates), '%Y-%m-%d')

Create list of all dates between first and last dates for columns of new dataframe

In [14]:
def get_all_dates(fd, ld):
    #returns list of all dates from firstdate fd to lastdate ld
    #fd and ld are strings in YYYY-MM-DD format
    
    monthlastday = {1: 31, 2: 29, 3: 31, 4: 30, 5: 31}
    
    fdmint = int(fd[5:7].lstrip('0'))
    ldmint = int(ld[5:7].lstrip('0'))
    
    monthlistint = list(range(fdmint, ldmint+1))
    
    datetimes = []
    
    for m in monthlistint:
        
        if m == fdmint:
            firstday = int(fd[8:10].lstrip('0'))
        else:
            firstday = 1
            
        if m == ldmint:
            lastday = int(ld[8:10].lstrip('0'))
        else:
            lastday = monthlastday[m]
        for d in range(firstday, lastday+1):

            datetimes.append('2020-' + datetime.strftime(datetime.strptime(str(m), '%m'), '%m') 
                           +'-' + datetime.strftime(datetime.strptime(str(d), '%d'), '%d') )
    
    return datetimes

columns = get_all_dates(firstdate, lastdate)
columns.insert(0, 'fips')

Create new dataframe with dates as columns. Use pandas dataframe append to take
care of matching values of rows by column name. Will take a few minutes.

In [15]:
#for each fip, append cases and deaths to transposed dataframes

fips = nyt['fips'].unique().tolist()
nfips = len(fips)

nyttransposedcases = pd.DataFrame(columns = columns)
nyttransposeddeaths = pd.DataFrame(columns = columns)

row = 1
for fip in fips:
    
    print('Working on fip ' + str(row) + '/' + str(nfips), end = "\r")
    
    tempcases = nyt.loc[nyt['fips'] == fip, ['date', 'cases']].set_index('date').T
    tempcases['fips'] = fip
    
    tempdeaths = nyt.loc[nyt['fips'] == fip, ['date', 'deaths']].set_index('date').T
    tempdeaths['fips'] = fip
    
    nyttransposedcases = nyttransposedcases.append(tempcases).reset_index(drop = True)
    nyttransposedcases.iloc[-1].fillna(0, axis = 0, inplace = True)
    
    nyttransposeddeaths = nyttransposeddeaths.append(tempdeaths).reset_index(drop = True)
    nyttransposeddeaths.iloc[-1].fillna(0, axis = 0, inplace = True)
    
    row = row + 1
    


Working on fip 13/2807

C:\Users\Franklin.000\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


New dataframes nyttransposedcases and nyttransposeddeaths

In [16]:
nyttransposedcases.head()

,2020-01-21,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,...,2020-04-17,2020-04-18,2020-04-19,2020-04-20,2020-04-21,2020-04-22,2020-04-23,2020-04-24,2020-04-25,fips
0,1,1,1,1,1,1,1,1,1,1,...,2055,2092,2142,2162,2162,2208,2239,2267,2297,53061
1,0,0,0,1,1,1,1,1,1,2,...,19391,20395,21272,22101,23181,24546,25811,27616,29058,17031
2,0,0,0,0,1,1,1,1,1,1,...,1501,1556,1636,1676,1691,1753,1827,1845,1969,6059
3,0,0,0,0,0,1,1,1,1,1,...,2404,2491,2589,2636,2738,2846,2970,3116,3234,4013
4,0,0,0,0,0,1,1,1,1,1,...,11391,12021,12341,13816,15140,16435,17508,18545,19107,6037


In [17]:
nyttransposeddeaths.head()

,2020-01-21,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,...,2020-04-17,2020-04-18,2020-04-19,2020-04-20,2020-04-21,2020-04-22,2020-04-23,2020-04-24,2020-04-25,fips
0,0,0,0,0,0,0,0,0,0,0,...,87,92,92,96,98,98,99,102,103,53061
1,0,0,0,0,0,0,0,0,0,0,...,760,860,877,915,1002,1072,1142,1220,1267,17031
2,0,0,0,0,0,0,0,0,0,0,...,28,32,32,33,33,34,36,36,38,6059
3,0,0,0,0,0,0,0,0,0,0,...,69,70,75,75,84,97,115,120,121,4013
4,0,0,0,0,0,0,0,0,0,0,...,495,576,600,617,663,729,797,850,895,6037
